# 計算台股強勢股比例

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import time

class StockAnalyzer:
    def __init__(self, stock_list_file, date_start, date_end, window_size):
        # 讀取股票清單
        self.data = pd.read_excel(stock_list_file)
        # 獲取代號
        self.target_stocks = self.data['代號'].tolist()
        # 將日期轉換成 datetime 物件
        self.date_start = date_start
        self.date_end = date_end
        self.window_size = window_size

    def calculate_strength_values(self, stock_data):
        # 取得收盤價資料
        stock_close = stock_data['Close']

        # 用來存放每組 window_size 天的股價漲跌幅
        stock_20days_returns = []

        # 遍歷每個滾動窗口的起始日期
        for i, group_index in enumerate(range(len(stock_close) - self.window_size + 1)):
            start_price = stock_close.iloc[group_index]
            end_price = stock_close.iloc[group_index + self.window_size - 1]
            stock_20days_return = ((end_price - start_price) / start_price) * 100

            # 將每組 window_size 天的漲跌幅加入列表
            stock_20days_returns.append(stock_20days_return)

        # 將漲跌幅列表轉換為 DataFrame
        return pd.DataFrame(stock_20days_returns, columns=['漲跌幅'])

    def analyze_stocks(self):
        # 下載台股加權指數的資料，progress=False不會顯示下載進度
        y_twii = "^TWII"
        twii_data = yf.download(y_twii, start=self.date_start, end=self.date_end, progress=False)

        # 計算台股加權指數強(弱)度值
        twii_strength_values = self.calculate_strength_values(twii_data)

        for target in self.target_stocks:
            # 獲取目標股票的代號
            stock_symbol = f"{target}.TW"
            # 傳入start跟end獲取歷史股價，progress=False不會顯示下載進度
            target_data = yf.download(stock_symbol, start=self.date_start, end=self.date_end, progress=False)
            # 使用 pandas 的數學運算計算強(弱)度值
            target_strength_values = self.calculate_strength_values(target_data)
            strength_values = ((target_strength_values['漲跌幅'] - twii_strength_values['漲跌幅']) / twii_strength_values['漲跌幅'])

            # 計算強勢股和弱勢股的比例
            positive_strength_ratio = (strength_values > 0).mean()
            negative_strength_ratio = 1 - positive_strength_ratio
            
            if positive_strength_ratio >= 0.6:
                print(f'Stock: {target} | 強勢股比例: {positive_strength_ratio * 100:.2f}%')

#                 #start_time = time.time()
#                 #if (target_data['Close'] <= 80).any() and (target_data['Close'] >= 60).any():
#                 if (target_data['Close'] <= 80).any():
#                    if (target_data['Close'] >= 60).any():
#                     print(f"Stock: {target} | 強勢股比例: {positive_strength_ratio * 100:.2f}% | 有股價低於80 " )

                #end_time = time.time()
                #print(f"判斷股價低於80時間: {end_time - start_time:.2f} 秒")

            #print(f'Stock: {target} | 強(弱)度值: {strength_values} | Status : Get!')
            #print(f'Stock: {target} | 強勢股比例: {positive_strength_ratio * 100:.2f}% | 弱勢股比例: {negative_strength_ratio * 100:.2f}%')


# 使用 StockAnalyzer 類別進行股票分析
stock_list_file = 'C:/Users/sheng/Desktop/F_Market/Trading/stock_list.xlsx'
date_start = '2023-07-01'
date_end = '2023-08-01'
window_size = 5

stock_analyzer = StockAnalyzer(stock_list_file, date_start, date_end, window_size)
stock_analyzer.analyze_stocks()


Stock: 1104 | 強勢股比例: 70.59%
Stock: 1453 | 強勢股比例: 76.47%
Stock: 1529 | 強勢股比例: 64.71%
Stock: 1560 | 強勢股比例: 64.71%
Stock: 1618 | 強勢股比例: 70.59%
Stock: 2009 | 強勢股比例: 64.71%
Stock: 2308 | 強勢股比例: 64.71%
Stock: 2317 | 強勢股比例: 82.35%
Stock: 2324 | 強勢股比例: 64.71%
Stock: 2328 | 強勢股比例: 64.71%
Stock: 2330 | 強勢股比例: 64.71%
Stock: 2344 | 強勢股比例: 70.59%
Stock: 2352 | 強勢股比例: 76.47%
Stock: 2353 | 強勢股比例: 64.71%
Stock: 2354 | 強勢股比例: 88.24%
Stock: 2358 | 強勢股比例: 64.71%
Stock: 2363 | 強勢股比例: 64.71%
Stock: 2382 | 強勢股比例: 64.71%
Stock: 2395 | 強勢股比例: 64.71%
Stock: 2397 | 強勢股比例: 64.71%
Stock: 2399 | 強勢股比例: 64.71%
Stock: 2401 | 強勢股比例: 64.71%
Stock: 2421 | 強勢股比例: 64.71%
Stock: 2451 | 強勢股比例: 70.59%
Stock: 2465 | 強勢股比例: 70.59%
Stock: 2495 | 強勢股比例: 64.71%
Stock: 2886 | 強勢股比例: 64.71%
Stock: 3004 | 強勢股比例: 64.71%
Stock: 3005 | 強勢股比例: 64.71%
Stock: 3022 | 強勢股比例: 64.71%
Stock: 3038 | 強勢股比例: 70.59%
Stock: 3046 | 強勢股比例: 64.71%
Stock: 3443 | 強勢股比例: 64.71%
Stock: 3447 | 強勢股比例: 64.71%
Stock: 3518 | 強勢股比例: 70.59%
Stock: 3532 | 強勢股比例:

# 測試執行時間

In [2]:
import timeit
from datetime import datetime
import yfinance as yf

date_start = datetime(2023, 7, 1)
date_end = datetime(2023, 8, 1)

# 定義兩個測試函數
def test_history():
    #獲取目標股票的Ticker類
    stock = yf.Ticker(f'{6405}.TW')
    #傳入start跟end獲取歷史股價
    df = stock.history(start=date_start, end=date_end)

def test_download():
    y_twii = '6405.TW'
    twii_data = yf.download(y_twii, start=date_start, end=date_end, progress=False)

# 測試執行時間
time_history = timeit.timeit(test_history, number=10)
time_download = timeit.timeit(test_download, number=10)

# 顯示結果
print("history 函數執行時間:", time_history)
print("download 函數執行時間:", time_download)

history 函數執行時間: 3.486505299999976
download 函數執行時間: 3.4605600999999524
